<div align="Center">

# JK Lakshmipat University
## Institute of Engineering and Technology
### Machine Learning (CS1138)
#### Project - 1B
#### RFM model-based Customer Segmentation using Clustering and Classification

</div>
<hr>

#### Importing the Libraries

In [ ]:
from calendar import month_abbr
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, MeanShift
from sklearn.neighbors import KNeighborsClassifier
from sklearn.manifold import TSNE
from sklearn import metrics

from scipy.cluster import hierarchy

from lifetimes import BetaGeoFitter, GammaGammaFitter
import lifetimes.utils as lfUtils
import lifetimes.plotting as lfPlt

<hr>

### Data Configuration

#### Importing the Data

In [ ]:
df1 = pd.read_excel('online_retail_II.xlsx', sheet_name='Year 2009-2010')
df2 = pd.read_excel('online_retail_II.xlsx', sheet_name='Year 2010-2011')
df = pd.concat([df1, df2])

#### Initial Dataset

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

#### Imputing the Dataset

In [ ]:
df.isnull().sum()

- Description : Not Available
- Customer ID : -1

In [ ]:
df['Description'] = df['Description'].fillna('Not Available')
df['Customer ID'] = df['Customer ID'].fillna(-1)

In [ ]:
df.drop_duplicates(keep='first', inplace=True)

#### Feature Engineering

In [ ]:
df['Customer ID'] = df['Customer ID'].astype(int)
df['TotalPrice'] = df['Price'] * df['Quantity']

In [ ]:
df['Country'] = df['Country'].astype('category')
df['Description'] = df['Description'].astype('category')
df['StockCode'] = df['StockCode'].astype(str)

In [ ]:
df[df['Price'] < 0]

In [ ]:
df['Cancelled'] = df['Invoice'].astype(str).str.contains('C').astype(int)
df['Bad Debt'] = df['Invoice'].astype(str).str.contains('A').astype(int)
df['Invoice'] = df['Invoice'].astype(str).str.replace('[A-Z]', '', regex=True).astype(int)

StockCode contains Codes for different Situations, so it Cannot be Converted to numerical.

In [ ]:
len(df['StockCode'].str.extractall(r"([a-zA-Z]+)").groupby(level=0).sum(numeric_only=False)[0].unique())

#### Final Dataset

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

#### Dataset with Valid Orders

In [ ]:
df2 = df.drop(df[(df['Bad Debt'] == 1) | (df['Cancelled'] == 1) | (df['Description'] == 'Not Available') | (df['Customer ID'] == -1) | (df['Price'] == 0.0) | ~(df['StockCode'].str.isdigit())].index)
df2.drop(['Bad Debt', 'Cancelled'], axis=1, inplace=True)

In [ ]:
df2.describe()

<hr>

### Exploratory Data Analysis

#### Top Selling Products

In [ ]:
plotDF = df.groupby('Description', observed=True).size().sort_values(ascending=False).reset_index()
plotDF.drop(plotDF[plotDF['Description'] == 'Not Available'].index, inplace=True)
plotDF.columns = ['Description', 'Count']

fig = px.bar(plotDF.head(20), x='Description', y='Count', title='Top Selling Products')
fig.show()

#### Most Expensive and Least Expensive Products

In [ ]:
plotDF = df2[['Description', 'Price']]
plotDF.sort_values(by='Price', ascending=False, inplace=True)
plotDF.drop_duplicates(subset=['Description'], keep='first', inplace=True)
plotDF.reset_index(drop=True, inplace=True)

In [ ]:
fig = px.line(plotDF, x='Description', y='Price', title='Prices of Offered Products')
fig.update_xaxes(showticklabels=False)
fig.show()

In [ ]:
fig = px.bar(plotDF.head(20), x='Description', y='Price', title='Most Expensive Products')
fig.show()

In [ ]:
fig = px.bar(plotDF.tail(20), x='Description', y='Price', title='Least Expensive Products')
fig.show()

#### Average Number of Orders per Customer

In [ ]:
ordersPerCustomer = df2[['Invoice', 'Customer ID']].drop_duplicates()
ordersPerCustomer = ordersPerCustomer.groupby(['Customer ID'], observed=True).count().reset_index().sort_values(by='Invoice', ascending=False).reset_index(drop=True)
ordersPerCustomer.columns = ['Customer ID', 'Number of Orders']

In [ ]:
print(f"Average Orders per Customer: {ordersPerCustomer['Number of Orders'].mean()}")

In [ ]:
plotDF = ordersPerCustomer.head(20)

fig = px.bar(plotDF, x=plotDF.index, y='Number of Orders', hover_data=['Customer ID'], title='Most Number of Orders per Customer')
fig.update_xaxes(title='Customer Rank')
fig.show()

#### Average Number of Unique Items per Customer and per Order

In [ ]:
itemsPerCustomer = df2[['Customer ID', 'Description']].drop_duplicates()
itemsPerCustomer = itemsPerCustomer.groupby(['Customer ID'], observed=True).count().reset_index().sort_values(by='Description', ascending=False).reset_index(drop=True)
itemsPerCustomer.columns = ['Customer ID', 'Number of Items']

itemsPerOrder = df2[['Invoice', 'Description']].drop_duplicates()
itemsPerOrder = itemsPerOrder.groupby(['Invoice'], observed=True).count().reset_index().sort_values(by='Description', ascending=False).reset_index(drop=True)
itemsPerOrder.columns = ['Invoice Number', 'Number of Items']

In [ ]:
print(f"Average Number of Unique Items per Customer: {itemsPerCustomer['Number of Items'].mean()}")
print(f"Average Number of Unique Items per Order: {itemsPerOrder['Number of Items'].mean()}")

In [ ]:
plotDF = itemsPerCustomer.head(20)

fig = px.bar(plotDF, x=plotDF.index, y='Number of Items', hover_data=['Customer ID'], title='Most Number of Unique Items per Customer')
fig.update_xaxes(title='Customer Rank')
fig.show()

In [ ]:
plotDF = itemsPerOrder.head(20)

fig = px.bar(plotDF, x=plotDF.index, y='Number of Items', hover_data=['Invoice Number'], title='Most Number of Unique Items per Order')
fig.update_xaxes(title='Order Rank')
fig.show()

#### Top Countries by Number of Customers and Number of Orders

In [ ]:
plotDF = df.groupby('Country', observed=True).size().sort_values(ascending=False).reset_index()
plotDF.columns = ['Country', 'Count']

fig = px.bar(plotDF, x='Country', y='Count', title='Sales per Country')
fig.show()

In [ ]:
plotDF = df[['Country','Customer ID']].drop_duplicates()
plotDF = plotDF.groupby(['Country'], observed=True)['Customer ID'].count().sort_values(ascending=False).reset_index()
plotDF.columns = ['Country', 'Count']

fig = px.bar(plotDF, x='Country', y='Count', title='Sales by Country per Customer')
fig.show()

In [ ]:
plotDF = df[['Country','Invoice']].drop_duplicates()
plotDF = plotDF.groupby(['Country'], observed=True)['Invoice'].count().sort_values(ascending=False).reset_index()
plotDF.columns = ['Country', 'Count']

fig = px.bar(plotDF, x='Country', y='Count', title='Sales by Country per Order')
fig.show()

#### Total Sales per Month

In [ ]:
plotDF = df2[['TotalPrice', 'InvoiceDate']]
plotDF = plotDF.groupby([plotDF['InvoiceDate'].dt.year, plotDF['InvoiceDate'].dt.month], observed=True).agg({'TotalPrice': 'sum'})
plotDF.reset_index(names=['Year', 'Month'], inplace=True)
plotDF['Month-Year'] = pd.Series(month_abbr[i] for i in plotDF['Month']) + ' ' + plotDF['Year'].astype(str)
plotDF.drop(['Year', 'Month'], axis=1, inplace=True)
plotDF.columns = ['Total Sales', 'Month & Year']

In [ ]:
fig = px.line(plotDF, x='Month & Year', y='Total Sales', title='Monthly Sales')
fig.show()

#### Cancelled Items Analysis

In [ ]:
plotDF = df[df['Cancelled'] == 1][['TotalPrice', 'InvoiceDate']]
plotDF = plotDF.groupby([plotDF['InvoiceDate'].dt.year, plotDF['InvoiceDate'].dt.month], observed=True).agg({'TotalPrice': ['sum', 'count']})
plotDF.reset_index(names=['Year', 'Month'], inplace=True)
plotDF['Month-Year'] = pd.Series(month_abbr[i] for i in plotDF['Month']) + ' ' + plotDF['Year'].astype(str)
plotDF.columns = ['Year', 'Month', 'Cancellation Amount', 'Total Cancellations', 'Month & Year']
plotDF.drop(['Year', 'Month'], axis=1, inplace=True)
plotDF['Cancellation Amount'] = - plotDF['Cancellation Amount']

In [ ]:
fig = px.line(plotDF, x='Month & Year', y='Total Cancellations', title='Monthly Cancellations', hover_data=['Cancellation Amount'])
fig.show()

<hr>

## Methodologies

<hr>

### RFM Analysis

#### Creating the RFM Table

In [ ]:
today = datetime.date(df2['InvoiceDate'].max())
print(f"The Last Date in the Dataset: {today.strftime('%d %B %Y')}")

In [ ]:
df2['Date'] = df2['InvoiceDate'].dt.date
recencyDF = df2[['Customer ID', 'Date']].drop_duplicates()
recencyDF = recencyDF.groupby(['Customer ID'], observed=True)['Date'].max().reset_index()
recencyDF.columns = ['Customer ID', 'Last Purchase Date']
recencyDF['Recency'] = today - recencyDF['Last Purchase Date']
recencyDF['Recency'] = recencyDF['Recency'].astype('timedelta64[ns]').astype('int64') // 86400000000000

In [ ]:
frequencyDF = df2[['Customer ID', 'Invoice']].drop_duplicates()
frequencyDF = frequencyDF.groupby(['Customer ID'], observed=True)['Invoice'].count().reset_index()
frequencyDF.columns = ['Customer ID', 'Frequency']

In [ ]:
monetaryDF = df2[['Customer ID', 'TotalPrice']]
monetaryDF = monetaryDF.groupby(['Customer ID'], observed=True)['TotalPrice'].sum().reset_index()
monetaryDF.columns = ['Customer ID', 'Monetary']

In [ ]:
rfmDF = pd.merge(recencyDF, frequencyDF, on='Customer ID')
rfmDF = pd.merge(rfmDF, monetaryDF, on='Customer ID')
rfmDF.drop('Last Purchase Date', axis=1, inplace=True)
rfmDF.set_index('Customer ID', inplace=True)

rfmDF.head()

#### Quantile Distribution

In [ ]:
fig = make_subplots(rows=1, cols=3)
fig.add_trace(go.Box(y=rfmDF['Recency'].values, name='Recency'), row=1, col=1)
fig.add_trace(go.Box(y=rfmDF['Frequency'].values, name='Frequency'), row=1, col=2)
fig.add_trace(go.Box(y=rfmDF['Monetary'].values, name='Monetary'), row=1, col=3)
fig.update_layout(title_text="Box Plot of RFM Values")
fig.show()

In [ ]:
def rScore(value: np.floating, category: str, quantiles: pd.DataFrame):
    "Decides a Customer's Score on the basis of Recency Value"
    if value <= quantiles[category][0.25]:
        return 4
    elif value <= quantiles[category][0.50]:
        return 3
    elif value <= quantiles[category][0.75]:
        return 2
    else:
        return 1

def fmScore(value: np.floating, category: str, quantiles: pd.DataFrame):
    "Decides a Customer's Score on the basis of Frequency/Monetary Value"
    if value <= quantiles[category][0.25]:
        return 1
    elif value <= quantiles[category][0.50]:
        return 2
    elif value <= quantiles[category][0.75]:
        return 3
    else:
        return 4

In [ ]:
rfmQuantiles = rfmDF[['Recency', 'Frequency', 'Monetary']].quantile(q=[0.25, 0.50, 0.75])
rfmQuantiles

In [ ]:
rfmSegmentation = rfmDF.copy()
rfmSegmentation['R'] = rfmSegmentation['Recency'].apply(rScore, args=('Recency', rfmQuantiles))
rfmSegmentation['F'] = rfmSegmentation['Frequency'].apply(fmScore, args=('Frequency', rfmQuantiles))
rfmSegmentation['M'] = rfmSegmentation['Monetary'].apply(fmScore, args=('Monetary', rfmQuantiles))
rfmSegmentation['RFM Score'] = rfmSegmentation['R'].astype(str) + rfmSegmentation['F'].astype(str) + rfmSegmentation['M'].astype(str)

rfmSegmentation.head()

#### Estimated Segmentation

In [ ]:
def rfmClusters(data: pd.DataFrame, rCol: str, fCol: str, mCol: str, rfmCol: str):
    """Segments the RFM Data into Clusters namely,

    Best Customers, Worst Customers, Newest/Most Loyal Customeres, Frequent Buyers, Big Spenders, and Others."""

    bestCustomers = data[data[rfmCol] == '444']
    worstCustomers = data[data[rfmCol] == '111']
    loyalCustomers = data[data[rCol] == 4]
    frequentBuyers = data[data[fCol] == 4]
    bigSpenders = data[data[mCol] == 4]

    others = data[(data[rfmCol] != '444') & (data[rfmCol] != '111') & (data[rCol] != 4) & (data[fCol] != 4) & (data[mCol] != 4)]

    return {
        'Best Customers': bestCustomers,
        'Worst Customers': worstCustomers,
        'Newest/Most Loyal Customers': loyalCustomers,
        'Frequent Buyers': frequentBuyers,
        'Big Spendors': bigSpenders,
        'Others': others
    }

In [ ]:
segments = rfmClusters(rfmSegmentation, 'R', 'F', 'M', 'RFM Score')

for segment in segments:
    print(f"{segment}: {segments[segment].shape[0]}")

In [ ]:
fig = px.pie(names=segments.keys(), values=[segments[segment].shape[0] for segment in segments], title='RFM Estimated Segmentation')
fig.show()

#### Standardization

In [ ]:
scaler = StandardScaler()
rfmScaled = scaler.fit_transform(rfmDF[['Recency', 'Frequency', 'Monetary']])
rfmScaled = pd.DataFrame(rfmScaled)
rfmScaled.columns = ['Recency', 'Frequency', 'Monetary']
rfmScaled.index = rfmDF.index
rfmScaled.head()

#### Normalization

In [ ]:
fig = px.imshow(rfmScaled.corr(), title='Correlation Matrix of RFM Values')
fig.show()

In [ ]:
fig = ff.create_scatterplotmatrix(rfmScaled, diag='histogram', height=500, width=1200, title='Scatter Matrix before Normalization')
fig.show()

In [ ]:
rfmNorm = pd.DataFrame({
    'Log Recency': np.log(rfmDF['Recency']+0.01),
    'Log Frequency': np.log(rfmDF['Frequency']),
    'Log Monetary': np.log(rfmDF['Monetary'])
})
rfmNorm.head()

In [ ]:
fig = px.imshow(rfmNorm.corr(), title='Correlation Matrix of Normalized RFM Values')
fig.show()

In [ ]:
fig = ff.create_scatterplotmatrix(rfmNorm, diag='histogram', height=500, width=1200, title='Scatter Matrix after Normalization')
fig.show()

<hr>

### k-Means Clustering

#### Selecting $k$

In [ ]:
kmcData = rfmNorm.to_numpy()
kRange = list(range(2, 11))
kData = list()

for i in kRange:
    model = KMeans(n_clusters=i, random_state=42)
    model.fit(kmcData)
    kData.append((model, model.inertia_/kmcData.shape[0], metrics.silhouette_score(kmcData, model.labels_), metrics.davies_bouldin_score(kmcData, model.labels_)))

In [ ]:
fig = make_subplots(rows=1, cols=3, subplot_titles=['Elbow Curve', 'Silhouette Scores', 'Davies Bouldin Scores'], x_title='Number of Clusters')
fig.add_trace(go.Scatter(x=kRange, y=[i[1] for i in kData], name='Elbow Curve'), row=1, col=1)
fig.add_trace(go.Scatter(x=kRange, y=[i[2] for i in kData], name='Silhouette Scores'), row=1, col=2)
fig.add_trace(go.Scatter(x=kRange, y=[i[3] for i in kData], name='Davies Bouldin Scores'), row=1, col=3)
fig.update_layout(title_text='Evaluating the Number of Clusters for k-Means Clustering', yaxis_title='Distortion', yaxis2_title='Silhouette Score')
fig.show()

#### Applying the Model

In [ ]:
kmcModel = kData[3] # 5 Clusters
rfmDF['Cluster kMC'] = kmcModel[0].labels_.astype(str)
rfmDF.head()

In [ ]:
fig = px.scatter_3d(rfmDF, x='Recency', y='Frequency', z='Monetary', color='Cluster kMC', symbol='Cluster kMC', opacity=0.5, title='RFM-based k-Means Clusters')
fig.show()

<hr>

### Hierarchical Clustering

#### Average Linkage

In [ ]:
hcMergings = hierarchy.linkage(rfmScaled, method='average', metric='minkowski')

In [ ]:
# # Only Use when Necessary
# fig = ff.create_dendrogram(hcMergings)
# fig.show('browser')

#### Creating Clusters

In [ ]:
hcLabels = hierarchy.cut_tree(hcMergings, 5).reshape(-1,)
rfmDF['Cluster HC'] = hcLabels.astype(str)
rfmDF.head()

In [ ]:
fig = px.scatter_3d(rfmDF, x='Recency', y='Frequency', z='Monetary', color='Cluster HC', symbol='Cluster HC', opacity=0.5, title='RFM-based Hierarchical Clusters')
fig.show()

<hr>

### Improving Hierarchical Clusters

#### Using t-SNE

In [ ]:
hcData = rfmScaled.to_numpy()
hcData_embedded = TSNE(metric='minkowski', init='random').fit_transform(hcData)

In [ ]:
hcData_embedded

#### Applying the Model

In [ ]:
hcMergings = hierarchy.linkage(hcData_embedded, method='average', metric='minkowski')

In [ ]:
# # Only Use when Necessary
# fig = ff.create_dendrogram(hcMergings)
# fig.show('browser')

In [ ]:
hcLabels = hierarchy.cut_tree(hcMergings, 5).reshape(-1,)
rfmDF['Cluster HC2'] = hcLabels.astype(str)
rfmDF.head()

#### Testing the Model

In [ ]:
rfmDF['Cluster HC'].value_counts()

In [ ]:
rfmDF['Cluster HC2'].value_counts()

In [ ]:
fig = px.scatter_3d(rfmDF, x='Recency', y='Frequency', z='Monetary', color='Cluster HC2', symbol='Cluster HC2', opacity=0.5, title='New Hierarchical Clusters')
fig.show()

<hr>

### k-NN Classification

#### Spliting Data into Train, Test and Validate

In [ ]:
knnDf = rfmScaled.copy()
knnDf['Cluster'] = rfmDF['Cluster HC2'].astype(int)

In [ ]:
knnTrain, knnTest = train_test_split(knnDf, test_size=0.2, random_state=1)
knnTrain, knnValidate = train_test_split(knnTrain, test_size=0.2, random_state=1)

In [ ]:
print(f"The Data was split as follows-\nTrain: {knnTrain.shape[0]}, Validate: {knnValidate.shape[0]}, Test: {knnTest.shape[0]}")

In [ ]:
X_train = knnTrain.drop('Cluster', axis=1)
y_train = knnTrain['Cluster']
X_validate = knnValidate.drop('Cluster', axis=1)
y_validate = knnValidate['Cluster']
X_test = knnTest.drop('Cluster', axis=1)
y_test = knnTest['Cluster']

#### Selecting $k$

In [ ]:
trainAccuracy = list()
validateAccuracy = list()

neighbours = tuple(range(1, 21))

for k in neighbours:
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train, y_train)
    trainAccuracy.append(model.score(X_train, y_train))
    validateAccuracy.append(model.score(X_validate, y_validate))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=neighbours, y=trainAccuracy, name='Train Accuracy'))
fig.add_trace(go.Scatter(x=neighbours, y=validateAccuracy, name='Validate Accuracy'))
fig.update_layout(title_text='Cross-Fold Validation for k-NN', xaxis_title='Number of Neighbours', yaxis_title='Accuracy')
fig.show()

#### Applying the Model

In [ ]:
knn = KNeighborsClassifier(n_neighbors=14)
knn.fit(X_train, y_train)
knn.fit(X_validate, y_validate)

#### Testing the Model

In [ ]:
y_pred = knn.predict(X_test)

In [ ]:
fig = px.imshow(metrics.confusion_matrix(y_test, y_pred), title='Confusion Matrix for k-NN Classification')
fig.show()

In [ ]:
print(metrics.classification_report(y_test, y_pred))

<hr>

### Mean-Shift Clustering

#### Applying the Model

In [ ]:
msc = MeanShift()
msc.fit(kmcData)

In [ ]:
rfmDF['Cluster MSC'] = msc.labels_.astype(str)
rfmDF.head()

In [ ]:
fig = px.scatter_3d(rfmDF, x='Recency', y='Frequency', z='Monetary', color='Cluster MSC', symbol='Cluster MSC', opacity=0.5, title='Mean-Shift Clusters')
fig.show()

<hr>

### Improving Mean-Shift Clusters

#### Using t-SNE

In [ ]:
mscData = rfmNorm.to_numpy()
mscData_embedded = TSNE(metric='minkowski', init='random').fit_transform(mscData)

In [ ]:
mscData_embedded

#### Applying The Model

In [ ]:
msc = MeanShift()
msc.fit(mscData_embedded)

In [ ]:
rfmDF['Cluster MSC2'] = msc.labels_.astype(str)
rfmDF.head()

#### Testing the Model

In [ ]:
rfmDF['Cluster MSC'].value_counts()

In [ ]:
rfmDF['Cluster MSC2'].value_counts()

In [ ]:
fig = px.scatter_3d(rfmDF, x='Recency', y='Frequency', z='Monetary', color='Cluster MSC', symbol='Cluster MSC2', opacity=0.5, title='Mean-Shift Clusters')
fig.show()

Failed Successfully

<hr>

### BG/NBD CLV Modelling
Beta-Geometric/Negative Binomial Distribution Customer Lifetime Value Modelling

#### Fitting the Model

In [ ]:
clvDF = df2[['Customer ID', 'TotalPrice', 'InvoiceDate']]
clvDF = lfUtils.summary_data_from_transaction_data(clvDF, 'Customer ID', 'InvoiceDate', monetary_value_col='TotalPrice', observation_period_end=today)
clvDF.columns = ['Frequency', 'Recency', 'Age', 'Monetary']
clvDF.head()

In [ ]:
bgf = BetaGeoFitter(penalizer_coef=0.0)
bgf.fit(clvDF['Frequency'], clvDF['Recency'], clvDF['Age'])
bgf.summary

#### Checking the Model Fit

In [ ]:
calibAndHoldoutDF = lfUtils.calibration_and_holdout_data(df2, 'Customer ID', 'InvoiceDate', calibration_period_end='2011-06-08', observation_period_end=today)
calibAndHoldoutDF.head()

In [ ]:
bgf.fit(calibAndHoldoutDF['frequency_cal'], calibAndHoldoutDF['recency_cal'], calibAndHoldoutDF['T_cal'])

lfPlt.plot_calibration_purchases_vs_holdout_purchases(bgf, calibAndHoldoutDF, figsize=(10, 6))
plt.show()

#### Model Metrics

In [ ]:
plt.figure(figsize=(10, 6))
lfPlt.plot_frequency_recency_matrix(bgf)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
lfPlt.plot_probability_alive_matrix(bgf)
plt.show()

In [ ]:
clvDF['Predicted Purchases'] = bgf.conditional_expected_number_of_purchases_up_to_time(1, clvDF['Frequency'], clvDF['Recency'], clvDF['Age'])
sortedByProb = clvDF.sort_values(by='Predicted Purchases', ascending=False)
sortedByProb.head()

In [ ]:
lfPlt.plot_period_transactions(bgf, figsize=(10, 6))
plt.show()

#### Customer Probability History

In [ ]:
bestTransactions = df2[df2['Customer ID'] == 12347] # sortedByProb.index[0] not used because of too much frequency

# lfPlt.plot_history_alive(bgf, 730, bestTransactions, 'InvoiceDate') # Gives Error due to having InvoiceDate as a Column and not Index

In [ ]:
path = [p[0] for p in lfUtils.calculate_alive_path(bgf, bestTransactions, 'InvoiceDate', 730, 'D')]
path_dates = pd.date_range(start=min(bestTransactions['InvoiceDate']), periods=len(path), freq='D')

customer_history = bestTransactions[['InvoiceDate']].copy()
customer_history.index = pd.DatetimeIndex(customer_history['InvoiceDate'])

customer_history['transactions'] = 1
customer_history.drop('InvoiceDate', axis=1, inplace=True)
customer_history = customer_history.resample('D').sum()

payment_dates = customer_history[customer_history['transactions'] >= 1].index

fig = px.line(x=path_dates, y=path, title='History of being Alive for Customer with ID 12347')
for dt in payment_dates.values:
    fig.add_vline(x=str(dt), line_dash='dash', line_color='red')
fig.update_layout(
    xaxis = dict(
        title = 'Date',
        range = [min(bestTransactions['InvoiceDate']), path_dates[-1]]
    ),
    yaxis = dict(
        title = 'Probability of being Alive',
        range = [0, 1.0],
        tickmode = 'array',
        tickvals = np.arange(0, 1.1, 0.1)
    )
)
fig.show()

<hr>

### Gamma-Gamma CLV Modelling

#### Fitting the Model

In [ ]:
returningCustomersLV = clvDF[clvDF['Frequency'] > 0]
returningCustomersLV.head()

In [ ]:
returningCustomersLV[['Monetary', 'Frequency']].corr()

In [ ]:
ggf = GammaGammaFitter(penalizer_coef = 0)
ggf.fit(returningCustomersLV['Frequency'], returningCustomersLV['Monetary'])
ggf.summary

#### Average Profit Estimation

In [ ]:
clvDF['Expected Avg Profit'] = ggf.conditional_expected_average_profit(clvDF['Frequency'], clvDF['Monetary'])
clvDF.head()

In [ ]:
print(f"Expected Conditional Average Profit: {clvDF['Expected Avg Profit'].mean()}\nAverage Profit: {clvDF[clvDF['Frequency'] > 0]['Monetary'].mean()}")

#### Customer Lifetime Value

In [ ]:
bgf.fit(clvDF['Frequency'], clvDF['Recency'], clvDF['Age'])

In [ ]:
clvDF['CLV'] = ggf.customer_lifetime_value(bgf, clvDF['Frequency'], clvDF['Recency'], clvDF['Age'], clvDF['Monetary'], time=12, discount_rate=0.01)
clvDF.head()

In [ ]:
plotDF = clvDF.sort_values(by='CLV', ascending=False).reset_index().head(20)
fig = px.bar(plotDF, y='CLV', hover_data=['Customer ID', 'Recency', 'Frequency', 'Monetary', 'Age'], title='Top 20 Customers by CLV')
fig.show()

<hr>

## Conclusion

#### Data

In [ ]:
conclusionDF = rfmDF.copy()
conclusionDF['Age'] = clvDF['Age']
conclusionDF['Predicted Purchases'] = clvDF['Predicted Purchases']
conclusionDF['Expected Avg Profit'] = clvDF['Expected Avg Profit']
conclusionDF['CLV'] = clvDF['CLV']

In [ ]:
conclusionDF.sort_values(by='CLV', ascending=False)

In [ ]:
conclusionDF.sort_values(by='Predicted Purchases', ascending=False)

In [ ]:
conclusionDF.sort_values(by='Expected Avg Profit', ascending=False)

#### Models' Metrics

##### k-Means Clustering Model with $k=5$

In [ ]:
kmcModel[0]

In [ ]:
print(f"Silhouette Score: {kmcModel[2]:.3f}")
print(f"Davies-Bouldin Index: {kmcModel[3]:.3f}")
print(f"Calinski-Harabasz Index: {metrics.calinski_harabasz_score(kmcData, kmcModel[0].labels_):.3f}")

##### Agglomerative Hierarchical Clustering Model with $k=5$

In [ ]:
hcMergings

In [ ]:
print(f"Silhouette Score: {metrics.silhouette_score(hcData_embedded, hcLabels):.3f}")
print(f"Davies-Bouldin Index: {metrics.davies_bouldin_score(hcData_embedded, hcLabels):.3f}")
print(f"Calinski-Harabasz Index: {metrics.calinski_harabasz_score(hcData_embedded, hcLabels):.3f}")

##### k-NN Classification Model with $k=14$

In [ ]:
knn

In [ ]:
print(metrics.classification_report(y_test, y_pred))

##### Mean-Shift Clustering Model

In [ ]:
msc

In [ ]:
print(f"Silhouette Score: {metrics.silhouette_score(mscData_embedded, msc.labels_):.3f}")
print(f"Davies-Bouldin Index: {metrics.davies_bouldin_score(mscData_embedded, msc.labels_):.3f}")
print(f"Calinski-Harabasz Index: {metrics.calinski_harabasz_score(mscData_embedded, msc.labels_):.3f}")

##### BG/NBD and Gamma-Gamma Modelling

In [ ]:
bgf

In [ ]:
ggf

#### Cluster Description

In [ ]:
print("In k-Means Clustering, the RFM Values are as follows-")
for cls in conclusionDF['Cluster kMC'].value_counts().index:
    myDF = conclusionDF[conclusionDF['Cluster kMC'] == cls][["Recency", "Frequency", "Monetary", "Age"]]
    print(f"For Cluster {cls}:\n{myDF.describe()}\n")

In [ ]:
print("In Hierarchical Clustering, the RFM Values are as follows-")
for cls in conclusionDF['Cluster HC2'].value_counts().index:
    myDF = conclusionDF[conclusionDF['Cluster HC2'] == cls][["Recency", "Frequency", "Monetary", "Age"]]
    print(f"For Cluster {cls}:\n{myDF.describe()}\n")

In [ ]:
print("In Mean-Shift Clustering, the RFM Values are as follows-")
for cls in conclusionDF['Cluster MSC2'].value_counts().index:
    myDF = conclusionDF[conclusionDF['Cluster MSC2'] == cls][["Recency", "Frequency", "Monetary", "Age"]]
    print(f"For Cluster {cls}:\n{myDF.describe()}\n")

In [ ]:
fig = make_subplots(rows=1, cols=3, x_title='Clusters')
fig.add_trace(go.Box(x=conclusionDF['Cluster kMC'], y=conclusionDF['Recency'], name='Recency'), row=1, col=1)
fig.add_trace(go.Box(x=conclusionDF['Cluster kMC'], y=conclusionDF['Frequency'], name='Frequency'), row=1, col=2)
fig.add_trace(go.Box(x=conclusionDF['Cluster kMC'], y=conclusionDF['Monetary'], name='Monetary'), row=1, col=3)
fig.update_layout(title_text='k-Means Clusters as per Recency, Frequency and Monetary Values', yaxis_title='Recency', yaxis2_title='Frequency', yaxis3_title='Monetary')
fig.show()

In [ ]:
plotDF = conclusionDF.groupby('Cluster kMC').agg({'Monetary': ['count', 'sum']}).reset_index()

fig = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]])
fig.add_trace(go.Pie(values=plotDF['Monetary']['count'], labels=plotDF['Cluster kMC'], name='No. of Customers'), row=1, col=1)
fig.add_trace(go.Pie(values=plotDF['Monetary']['sum'], labels=plotDF['Cluster kMC'], name='Associated Sales'), row=1, col=2)
fig.update_layout(title_text='Customer Segmentation and Associated Sales as per k-Means Clusters')
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=3, x_title='Clusters')
fig.add_trace(go.Box(x=conclusionDF['Cluster HC2'], y=conclusionDF['Recency'], name='Recency'), row=1, col=1)
fig.add_trace(go.Box(x=conclusionDF['Cluster HC2'], y=conclusionDF['Frequency'], name='Frequency'), row=1, col=2)
fig.add_trace(go.Box(x=conclusionDF['Cluster HC2'], y=conclusionDF['Monetary'], name='Monetary'), row=1, col=3)
fig.update_layout(title_text='New Hierarchical Clusters as per Recency, Frequency and Monetary Values', yaxis_title='Recency', yaxis2_title='Frequency', yaxis3_title='Monetary')
fig.show()

In [ ]:
plotDF = conclusionDF.groupby('Cluster HC2').agg({'Monetary': ['count', 'sum']}).reset_index()

fig = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]])
fig.add_trace(go.Pie(values=plotDF['Monetary']['count'], labels=plotDF['Cluster HC2'], name='No. of Customers'), row=1, col=1)
fig.add_trace(go.Pie(values=plotDF['Monetary']['sum'], labels=plotDF['Cluster HC2'], name='Associated Sales'), row=1, col=2)
fig.update_layout(title_text='Customer Segmentation and Associated Sales as per k-Means Clusters')
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=3, x_title='Clusters')
fig.add_trace(go.Box(x=conclusionDF['Cluster MSC2'], y=conclusionDF['Recency'], name='Recency'), row=1, col=1)
fig.add_trace(go.Box(x=conclusionDF['Cluster MSC2'], y=conclusionDF['Frequency'], name='Frequency'), row=1, col=2)
fig.add_trace(go.Box(x=conclusionDF['Cluster MSC2'], y=conclusionDF['Monetary'], name='Monetary'), row=1, col=3)
fig.update_layout(title_text='New Mean-Shift Clusters as per Recency, Frequency and Monetary Values', yaxis_title='Recency', yaxis2_title='Frequency', yaxis3_title='Monetary')
fig.show()

In [ ]:
plotDF = conclusionDF.groupby('Cluster MSC2').agg({'Monetary': ['count', 'sum']}).reset_index()

fig = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]])
fig.add_trace(go.Pie(values=plotDF['Monetary']['count'], labels=plotDF['Cluster MSC2'], name='No. of Customers'), row=1, col=1)
fig.add_trace(go.Pie(values=plotDF['Monetary']['sum'], labels=plotDF['Cluster MSC2'], name='Associated Sales'), row=1, col=2)
fig.update_layout(title_text='Customer Segmentation and Associated Sales as per k-Means Clusters')
fig.show()

<hr>